In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

SMALL_SIZE = 18
MEDIUM_SIZE = 20
BIGGER_SIZE = 22

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

dataframe = pd.read_csv("data/slr03.csv")
data = dataframe.values[:,:]
sand_granules_diameter = data[:,0]
beach_slope = data[:,1]

def plot_data(axis):
    global sand_granules_diameter, beach_slope
    axis.set_xlabel("sand granules diameter [mm]")
    axis.set_ylabel("beach slope [$^\circ$]")
    axis.plot(sand_granules_diameter, beach_slope, "ko")  
    axis.set_ylim(axis.get_ylim()[0], 17)
    
def plot_regression(axis, should_plot_error, a = 1, b = 1, relative_regression_error_a = "NA"):
    global sand_granules_diameter, beach_slope
    if relative_regression_error_a is not "NA":
        a, b = np.polyfit(sand_granules_diameter, beach_slope, 1)
        a = (1 + relative_regression_error_a)*a
    beach_slope_estimated = [a*i + b for i in sand_granules_diameter]
    linear_equation_as_string = "$\hat{y}$ = " + "{:.2f}".format(a) +" * $x$ " + "{:.2f}".format(b)
    axis.text(sand_granules_diameter[-1], b, linear_equation_as_string, color="blue", ha="right", va="bottom", size=18)
    quadratic_error = 0
    if should_plot_error:
            for i in range(0,len(sand_granules_diameter)):
                axis.plot([sand_granules_diameter[i],sand_granules_diameter[i]], [beach_slope[i],beach_slope_estimated[i]], "r")
                quadratic_error = calculate_quadratic_error(y=beach_slope, y_estimated=beach_slope_estimated)
                quadratic_error_equation_as_string = "$e = \sum_{i=0}^n (y_i - \hat{y}_i)^2$ = " + "{:.2f}".format(quadratic_error)
                axis.text(sand_granules_diameter[-1], b+1, quadratic_error_equation_as_string, color="red", ha="right", va="bottom", size=18)
    sand_granules_diameter_temp = np.insert(sand_granules_diameter,0,0)   
    beach_slope_estimated = np.insert(beach_slope_estimated,0,b)
    axis.plot(sand_granules_diameter_temp, beach_slope_estimated, "b")
    axis.set_ylim(axis.get_ylim()[0], 17)
    return (a, b, quadratic_error)
    
def calculate_quadratic_error(y, y_estimated):
    return np.sum((y-y_estimated)**2)

def plot_error_graph(axis):
    global sand_granules_diameter, beach_slope
    axis.set_xlabel("line slope $a$ [$^\circ$/mm]")
    axis.set_ylabel("sum of squared errors $e$ [$^{\circ 2}$]")
    a,b = np.polyfit(sand_granules_diameter, beach_slope, 1)
    a_s = []
    quadratic_errors = []
    for i in range(-35,36):
        a_temp = ((100.0+float(i))/100.0) * a
        beach_slope_estimated = [a_temp*i + b for i in sand_granules_diameter]
        quadratic_error = calculate_quadratic_error(y=beach_slope, y_estimated=beach_slope_estimated)
        a_s.append(a_temp)
        quadratic_errors.append(quadratic_error)
    axis.plot(a_s, quadratic_errors, "gray")
    c, d, e = np.polyfit(a_s, quadratic_errors, 2)
    return (c, d, e)
    
def plot_error_point(axis, a, quadratic_error):
    y_lim_bottom = axis.get_ylim()[0]
    x_lim_bottom = axis.get_xlim()[0]
    axis.plot([a, a], [y_lim_bottom, quadratic_error], "b")
    axis.plot([x_lim_bottom, a], [quadratic_error, quadratic_error], "r")
    axis.plot(a, quadratic_error, "o", color="gray")
    axis.set_ylim(y_lim_bottom, axis.get_ylim()[1])
    axis.set_xlim(x_lim_bottom, 37)
    
def calculate_gradient_of_parabola(a, c, d):
    return 2*c*a + d
    
def plot_gradient_of_parabola(axis, a, quadratic_error, gradient):
    x_lim_bottom = axis.get_xlim()[0]
    axis.annotate("", xy=(a + gradient, quadratic_error), xycoords='data', xytext=(a, quadratic_error), textcoords='data', arrowprops=dict(arrowstyle="->", connectionstyle="arc3", color="black"))
    axis.set_xlim(x_lim_bottom, 37)
    

In [2]:
fig, ax = plt.subplots(figsize=(8, 8))
plot_data(
    axis=ax
)
fig.savefig("images/plot1.png", bbox_inches='tight')

# How steap is your shore?
---
![](images/plot1.pdf)
<sub><sup>Source: http://college.cengage.com/mathematics/brase/understandable_statistics/7e/students/datasets/</sup></sub>

In [3]:
fig, ax = plt.subplots(figsize=(8, 8))
plot_regression(
    axis=ax,
    relative_regression_error_a=0.0,
    should_plot_error=False
)
plot_data(
    axis=ax
)
fig.savefig("images/plot2.png", bbox_inches='tight')

# How steap is your shore?
---
![](images/plot2.pdf)

In [4]:
fig, ax = plt.subplots(figsize=(8, 8))
plot_regression(
    axis=ax,
    relative_regression_error_a=0.0,
    should_plot_error=True
)
plot_data(
    axis=ax
)
fig.savefig("images/plot3.png", bbox_inches='tight')

# How steap is your shore?
---
![](images/plot3.pdf) 

In [5]:
relative_regression_error_a=0.3
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
a, b, quadratic_error = plot_regression(
    axis=ax1,
    relative_regression_error_a=relative_regression_error_a,
    should_plot_error=True
)
plot_data(
    axis=ax1
)
c, d, e = plot_error_graph(
    axis=ax2
)
plot_error_point(
    axis=ax2,
    a=a,
    quadratic_error=quadratic_error
)
fig.savefig("images/plot4.png", bbox_inches='tight')

# Always go downhill
---
![](images/plot4.png) 

In [6]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
a, b, quadratic_error = plot_regression(
    axis=ax1,
    relative_regression_error_a=relative_regression_error_a,
    should_plot_error=True
)
plot_data(
    axis=ax1
)
c, d, e = plot_error_graph(
    axis=ax2
)
gradient = calculate_gradient_of_parabola(
    a=a,
    c=c,
    d=d
)
plot_gradient_of_parabola(
    axis=ax2,
    a=a,
    quadratic_error=quadratic_error,
    gradient = gradient
)
plot_error_point(
    axis=ax2,
    a=a,
    quadratic_error=quadratic_error
)
ax2.text((2*a + gradient)/2, quadratic_error, "$e'(22.31)$ = " + "{:.2f}".format(gradient), color="black", ha="center", va="bottom", size=18)
fig.savefig("images/plot5.png", bbox_inches='tight')

# Always go downhill
---
![](images/plot5.png) 

In [7]:
step_length = 0.1

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
a, b, quadratic_error = plot_regression(
    axis=ax1,
    relative_regression_error_a=relative_regression_error_a,
    should_plot_error=True
)
plot_data(
    axis=ax1
)
c, d, e = plot_error_graph(
    axis=ax2
)
gradient = calculate_gradient_of_parabola(
    a=a,
    c=c,
    d=d
)
plot_gradient_of_parabola(
    axis=ax2,
    a=a,
    quadratic_error=quadratic_error,
    gradient = gradient
)
plot_gradient_of_parabola(
    axis=ax2,
    a=a,
    quadratic_error=quadratic_error,
    gradient = -step_length*gradient
)
plot_error_point(
    axis=ax2,
    a=a,
    quadratic_error=quadratic_error
)
ax2.text((2*a + gradient)/2, quadratic_error, "$e'$(" + "{:.2f}".format(a) + ") = " + "{:.2f}".format(gradient), color="black", ha="center", va="bottom", size=18)
ax2.text((2*a - step_length*gradient)/2, quadratic_error, "{:.2f}".format(-step_length*gradient), color="black", ha="center", va="bottom", size=18)
fig.savefig("images/plot6.png", bbox_inches='tight')
a = a - step_length*gradient

# Always go downhill
---
![](images/plot6.png) 

In [8]:
steps = 20
filenames = ["images/plot6.png"]

for i in range(steps):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
    _, _, quadratic_error = plot_regression(
        axis=ax1,
        a=a,
        b=b,
        should_plot_error=True
    )
    plot_data(
        axis=ax1
    )
    c, d, e = plot_error_graph(
        axis=ax2
    )
    gradient = calculate_gradient_of_parabola(
        a=a,
        c=c,
        d=d
    )
    plot_gradient_of_parabola(
        axis=ax2,
        a=a,
        quadratic_error=quadratic_error,
        gradient = -step_length*gradient
    )
    plot_error_point(
        axis=ax2,
        a=a,
        quadratic_error=quadratic_error
    )
    ax2.text((2*a - step_length*gradient)/2, quadratic_error, "{:.2f}".format(-step_length*gradient), color="black", ha="center", va="bottom", size=18)
    filename = "images/gif" + str(i) + ".png"
    filenames.append(filename)
    fig.savefig(filename, bbox_inches='tight')
    a = a - step_length*gradient
    
import sys
import datetime
import imageio

def create_gif(filenames, duration):
    images = []
    for filename in filenames:
        images.append(imageio.imread(filename))
    output_file = "images/gradient_descent.gif"
    imageio.mimsave(output_file, images, duration=duration)
    
create_gif(filenames=filenames, duration=1)
plt.close("all")

/usr/local/lib/python2.7/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


# Always go downhill
---
![](images/gradient_descent.gif) 

# We were promised Machine Learning!
---
<img src="images/linear_neural_network.pdf" alt="Drawing" style="width: 600px;"/>

In [9]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD

Using TensorFlow backend.


In [10]:
model = Sequential([
    Dense(1, input_dim=1, activation="linear", weights=[np.array([[22.31]]),np.array([-2.48])])
])
model.compile(loss='mse', optimizer=SGD(lr=0.1))
model.fit(sand_granules_diameter, beach_slope, epochs=1000)
print(model.get_weights())

Epoch 1/1000
9/9 [==============================] - 0s - loss: 5.0644
Epoch 2/1000
9/9 [==============================] - 0s - loss: 3.8341
Epoch 3/1000
9/9 [==============================] - 0s - loss: 3.0861
Epoch 4/1000
9/9 [==============================] - 0s - loss: 2.6295
Epoch 5/1000
9/9 [==============================] - 0s - loss: 2.3491
Epoch 6/1000
9/9 [==============================] - 0s - loss: 2.1753
Epoch 7/1000
9/9 [==============================] - 0s - loss: 2.0660
Epoch 8/1000
9/9 [==============================] - 0s - loss: 1.9957
Epoch 9/1000
9/9 [==============================] - 0s - loss: 1.9490
Epoch 10/1000
9/9 [==============================] - 0s - loss: 1.9167
Epoch 11/1000
9/9 [==============================] - 0s - loss: 1.8931
Epoch 12/1000
9/9 [==============================] - 0s - loss: 1.8749
Epoch 13/1000
9/9 [==============================] - 0s - loss: 1.8599
Epoch 14/1000
9/9 [==============================] - 0s - loss: 1.8470
Epoch 15/1000
9

Epoch 230/1000
9/9 [==============================] - 0s - loss: 1.1747
Epoch 231/1000
9/9 [==============================] - 0s - loss: 1.1742
Epoch 232/1000
9/9 [==============================] - 0s - loss: 1.1738
Epoch 233/1000
9/9 [==============================] - 0s - loss: 1.1733
Epoch 234/1000
9/9 [==============================] - 0s - loss: 1.1729
Epoch 235/1000
9/9 [==============================] - 0s - loss: 1.1725
Epoch 236/1000
9/9 [==============================] - 0s - loss: 1.1720
Epoch 237/1000
9/9 [==============================] - 0s - loss: 1.1716
Epoch 238/1000
9/9 [==============================] - 0s - loss: 1.1712
Epoch 239/1000
9/9 [==============================] - 0s - loss: 1.1708
Epoch 240/1000
9/9 [==============================] - 0s - loss: 1.1704
Epoch 241/1000
9/9 [==============================] - 0s - loss: 1.1700
Epoch 242/1000
9/9 [==============================] - 0s - loss: 1.1696
Epoch 243/1000
9/9 [==============================] - 0s - loss:

9/9 [==============================] - 0s - loss: 1.1436
Epoch 458/1000
9/9 [==============================] - 0s - loss: 1.1436
Epoch 459/1000
9/9 [==============================] - 0s - loss: 1.1436
Epoch 460/1000
9/9 [==============================] - 0s - loss: 1.1435
Epoch 461/1000
9/9 [==============================] - 0s - loss: 1.1435
Epoch 462/1000
9/9 [==============================] - 0s - loss: 1.1435
Epoch 463/1000
9/9 [==============================] - 0s - loss: 1.1435
Epoch 464/1000
9/9 [==============================] - 0s - loss: 1.1435
Epoch 465/1000
9/9 [==============================] - 0s - loss: 1.1435
Epoch 466/1000
9/9 [==============================] - 0s - loss: 1.1434
Epoch 467/1000
9/9 [==============================] - 0s - loss: 1.1434
Epoch 468/1000
9/9 [==============================] - 0s - loss: 1.1434
Epoch 469/1000
9/9 [==============================] - 0s - loss: 1.1434
Epoch 470/1000
9/9 [==============================] - 0s - loss: 1.1434
Epoch 4

Epoch 571/1000
9/9 [==============================] - 0s - loss: 1.1426
Epoch 572/1000
9/9 [==============================] - 0s - loss: 1.1426
Epoch 573/1000
9/9 [==============================] - 0s - loss: 1.1426
Epoch 574/1000
9/9 [==============================] - 0s - loss: 1.1426
Epoch 575/1000
9/9 [==============================] - 0s - loss: 1.1426
Epoch 576/1000
9/9 [==============================] - 0s - loss: 1.1425
Epoch 577/1000
9/9 [==============================] - 0s - loss: 1.1425
Epoch 578/1000
9/9 [==============================] - 0s - loss: 1.1425
Epoch 579/1000
9/9 [==============================] - 0s - loss: 1.1425
Epoch 580/1000
9/9 [==============================] - 0s - loss: 1.1425
Epoch 581/1000
9/9 [==============================] - 0s - loss: 1.1425
Epoch 582/1000
9/9 [==============================] - 0s - loss: 1.1425
Epoch 583/1000
9/9 [==============================] - 0s - loss: 1.1425
Epoch 584/1000
9/9 [==============================] - 0s - loss:

9/9 [==============================] - 0s - loss: 1.1423
Epoch 799/1000
9/9 [==============================] - 0s - loss: 1.1423
Epoch 800/1000
9/9 [==============================] - 0s - loss: 1.1423
Epoch 801/1000
9/9 [==============================] - 0s - loss: 1.1423
Epoch 802/1000
9/9 [==============================] - 0s - loss: 1.1423
Epoch 803/1000
9/9 [==============================] - 0s - loss: 1.1423
Epoch 804/1000
9/9 [==============================] - 0s - loss: 1.1423
Epoch 805/1000
9/9 [==============================] - 0s - loss: 1.1423
Epoch 806/1000
9/9 [==============================] - 0s - loss: 1.1423
Epoch 807/1000
9/9 [==============================] - 0s - loss: 1.1423
Epoch 808/1000
9/9 [==============================] - 0s - loss: 1.1423
Epoch 809/1000
9/9 [==============================] - 0s - loss: 1.1423
Epoch 810/1000
9/9 [==============================] - 0s - loss: 1.1423
Epoch 811/1000
9/9 [==============================] - 0s - loss: 1.1423
Epoch 8

Epoch 912/1000
9/9 [==============================] - 0s - loss: 1.1423
Epoch 913/1000
9/9 [==============================] - 0s - loss: 1.1423
Epoch 914/1000
9/9 [==============================] - 0s - loss: 1.1423
Epoch 915/1000
9/9 [==============================] - 0s - loss: 1.1423
Epoch 916/1000
9/9 [==============================] - 0s - loss: 1.1423
Epoch 917/1000
9/9 [==============================] - 0s - loss: 1.1423
Epoch 918/1000
9/9 [==============================] - 0s - loss: 1.1423
Epoch 919/1000
9/9 [==============================] - 0s - loss: 1.1423
Epoch 920/1000
9/9 [==============================] - 0s - loss: 1.1423
Epoch 921/1000
9/9 [==============================] - 0s - loss: 1.1423
Epoch 922/1000
9/9 [==============================] - 0s - loss: 1.1423
Epoch 923/1000
9/9 [==============================] - 0s - loss: 1.1423
Epoch 924/1000
9/9 [==============================] - 0s - loss: 1.1423
Epoch 925/1000
9/9 [==============================] - 0s - loss:

# How to model non-linear correlations?
---
<img src="images/mnist_examples.png" alt="Drawing" style="width: 600px;"/>
<sub><sup>Source: http://blog.welcomege.com/mnist-database/</sup></sub>

# Just increase the number of neurons
---
<img src="images/neural_network.png" alt="Drawing" style="width: 600px;"/>
<sub><sup>Source: http://neuralnetworksanddeeplearning.com/chap1.html</sup></sub>

# Just increase the number of neurons
---
<img src="images/neural_network.pdf" alt="Drawing" style="width: 1300px;"/>
<sub><sup>Source: http://neuralnetworksanddeeplearning.com/chap1.html</sup></sub>

In [11]:
from keras.utils import np_utils
from keras.datasets import mnist
 
(X_train, y_train), (X_test, y_test) = mnist.load_data()
 
X_train = X_train.reshape(X_train.shape[0], 784)
X_test = X_test.reshape(X_test.shape[0], 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
 
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

# And train again!
---

In [12]:
model = Sequential([
    Dense(15, input_dim=784, activation="linear"),
    Dense(10, activation="softmax")
])
model.compile(loss='mse', optimizer=SGD(lr=0.1), metrics=["accuracy"])
model.fit(X_train, Y_train, epochs=20)
model.evaluate(X_test, Y_test)

Epoch 1/20
60000/60000 [==============================] - 2s - loss: 0.0448 - acc: 0.7076     
Epoch 2/20
60000/60000 [==============================] - 2s - loss: 0.0212 - acc: 0.8708     
Epoch 3/20
60000/60000 [==============================] - 2s - loss: 0.0174 - acc: 0.8909     
Epoch 4/20
60000/60000 [==============================] - 2s - loss: 0.0157 - acc: 0.9005     
Epoch 5/20
60000/60000 [==============================] - 2s - loss: 0.0148 - acc: 0.9065     
Epoch 6/20
60000/60000 [==============================] - 2s - loss: 0.0141 - acc: 0.9100     
Epoch 7/20
60000/60000 [==============================] - 2s - loss: 0.0136 - acc: 0.9128     
Epoch 8/20
60000/60000 [==============================] - 2s - loss: 0.0133 - acc: 0.9156     
Epoch 9/20
60000/60000 [==============================] - 2s - loss: 0.0130 - acc: 0.9168     
Epoch 10/20
60000/60000 [==============================] - 2s - loss: 0.0128 - acc: 0.9179     
Epoch 11/20
60000/60000 [========================

[0.011377410997578408, 0.92679999999999996]